In [15]:
import warnings
warnings.filterwarnings('ignore')
import json
import csv
import re
import time
import sqlite3
import requests
from urllib.request import URLError
from pandas import DataFrame
from bs4 import BeautifulSoup
from multiprocessing import Pool

total_user = set()
def merge_dicts(*dict_args): #合併多個dictionary
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

def GetPageContent(url):
    try:
        #headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36 Edg/91.0.864.37'}
        headers = {
            
            'accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
            #'Host': 'https://www.mobile01.com/topiclist.php?f=566',
            'sec-fetch-dest': 'image',
            'sec-fetch-mode': 'no-cors',
            'sec-fetch-site': 'cross-site',
            "Upgrade-Insecure-Requests": "1", 
            
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'
            
            
        }
        res = requests.get(url, headers=headers)
        content = BeautifulSoup(res.text, "html.parser")
    except URLError as e:
        print(e)
    return content

def GetArticleTitle(url):
    content = GetPageContent('https://www.mobile01.com/' + url)
    Article_title = content.head.title.string
    Article_title = Article_title.replace(' - Mobile01',"")
    print(Article_title)
    return Article_title
# 取得每個網頁的總頁數
def GetTotalPageNum(url):
    content = GetPageContent('https://www.mobile01.com/' + url)
    pagination = content.select('a.c-pagination')# "[1][2][3]...下一頁"這行    
    
    if pagination:
        total_page = pagination[-1].string#最後一個值為總頁數
    else:
        total_page = 1
    print(total_page)
    return int(total_page)

# 取得從第 1 頁到第 page_num 頁的所有文章網址
def GetPosts(page_num, url):
    posts = list()
    
    for i in range(1, page_num): #
        content = GetPageContent('https://www.mobile01.com/' + url + '&p=' + str(i))
        
        for i in content.find_all("div",class_="l-listTable__tr"): #刪除置頂文章           
            if i.span: 
                if i.span.string == "置頂":
                    i.extract()
        
        rows = content.find_all("div",class_="c-listTableTd__title") #從文章title找網址    
         #rows = content.find_all("div",class_="l-articleCardDesc")
        for row in rows: #加入文章網址
            if row.a["href"] != None:    
                posts.append(row.a["href"])
    print("posts成功",posts)
    return posts

#得到使用者的評論,id,資訊
def GetArticleComment(url,head_title):
    user_comment = []
    total_page = GetTotalPageNum(url)       
    Article_title = GetArticleTitle(url)
    
    for j in range(1,1 + total_page): #文章中(1~最後一頁)的網址        
        content = GetPageContent('https://www.mobile01.com/' + url + '&p=' + str(j)) #文章中每一頁網址
        Article = content.find_all("div",class_="l-articlePage") #找到文章中的每個評論資訊
       
           
            
        for i in Article: #從第一個到最後一個評論
            if i.find("a",href=re.compile('^(userinfo.php)')) != None:
                str1 = ""
                user_id_dict = {'user_id':'','user_comment':[]}
                user_info_web = i.find("a",href=re.compile('^(userinfo.php)')) #使用者個資的網址
                user_id = i.find("div",class_="l-interact__pic") 
                user_id = user_id.img["alt"]
                

                
                user_content = i.find("article") #使用者id                 
                for text in user_content.find_all('blockquote'): #刪除"blockquote"標籤中的文字(回覆某個人的評論多出的文字)
                    if text != None:
                        text.extract()

                for x in user_content.stripped_strings: #使用者的評論
                    str1 = str1 + x
                
                WriteToCsv("htc_comment3",str1)
                user_dict = user_id
                #title_dict = user_comment
                if user_id not in total_user: #判斷user_id是否重複
                    globals()[user_dict]={"user_id":"","user_comment":[]}
                    WriteToCsv("htc_total_user_id",user_id)
                    user_info = GetUserInformation(user_info_web["href"],user_id) #傳送使用者網址回傳得到使用者個資
                    total_user.add(user_id)
                    globals()[user_dict]['user_id'] = user_id
                    #globals()[user_dict]['article_title'].append(head_title+':'+Article_title)
                    str1 = str1.split('。')
                    #for i in str1:
                    globals()[user_dict]['user_comment'].append(head_title+':'+str1)
                    globals()[user_dict] = merge_dicts(globals()[user_dict],user_info)
                    #print("沒有重複:",user_id)
                    #print(globals()[user_dict])
                    WriteToJson('user_id_article_title_info_comment2',globals()[user_dict])
                else:
                    if Article_title not in globals()[user_dict]['article_title']:
                        #globals()[user_dict]['article_title'].append(head_title+':'+Article_title)
                        globals()[user_dict]['user_comment'].append(head_title+':'+str1)
                    else:
                        globals()[user_dict]['user_comment'].append(head_title+':'+str1)
                    #print("重複:",user_id)
                    #print(globals()[user_dict])
                    #WriteToJson('user_id_article_title_info_comment2',globals()[user_dict])
            
    

    return 0

#抓取使用者的資訊
def GetUserInformation(url,user_id):
    dic = {'user_info':{'手機':0,'相機':0,'筆電':0,'電腦':0,'蘋果':0,'影音':0,'汽車':0,'機車':0,'單車':0,'遊戲':0,'居家':0,'女性':0,'時尚':0,'運動':0,'戶外':0,'生活':0,'旅遊':0,'閒聊':0,'時事':0,'市集':0,'悅遊日本':0}}
    content = GetPageContent('https://www.mobile01.com/' + url)
    p = re.compile('var percent_data = (.*);')  #使用正規化方法找var percent_data中的內容
    q = re.compile('var percent_lables = (.*);') #使用正規化方法找var percent_lables中的內容
    for script in content.find_all("script", {"src":False}):        
        if script != None:                   
            m = p.search(script.string) #使用正規化方法找var percent_data中的內容
            n = q.search(script.string) #使用正規化方法找var percent_lables中的內容
            if m and n:             
                str_m = m[1] #var percent_data中的內容
                str_m = (str_m[1:len(str_m)-1])
                str_n = n[1] #var percent_lables中的內容
                str_n = (str_n[1:len(str_n)-1])                
                list_m = str_m.split(",")
                list_n = str_n.split(",")                
                for i in range (0,len(list_m)):                    
                    test = list_n[i].replace('\\\\','\\') #將內容改成unicode的格式
                    test = test.replace("\"","") #將內容改成unicode的格式                
                    feature = test.encode('latin-1').decode('unicode_escape') #將var percent_lables中的內容轉換成中文
                    percent = list_m[i].replace('"',"")
                    dic['user_info'][feature] = percent #建立user_info dic)                                                                   
    return dic #回傳user_info dic

#寫入json檔
def WriteToJson(json_name,data):
    with open(json_name, 'a', encoding ='utf-8') as f: 
        json.dump(data, f,ensure_ascii = False)
    f.close()

def WriteToCsv(csv_name,data):
    with open(csv_name, 'a', encoding ='utf-8', newline='') as csvfile:
        csvfile.write(data)
        csvfile.write('\r\n')
    csvfile.close()

def main():
    print('start')
    result = []
    url = "topiclist.php?f=566"
    head_title = GetArticleTitle(url)
    total_pages = GetTotalPageNum(url)
    posts = GetPosts(total_pages, url)
    
    for i in posts:
        result1 = GetArticleComment(i,head_title)
    
    for i in total_user:
        result.append(globals()[i])
    WriteToJson("htc",result)   
    print('end')
if __name__ == '__main__':
    main()

start
Just a moment...
1
posts成功 []
end
